<h1>Training from Scratch with Various Architectures</h1>

This notebook can be used as a template for training models with a variety of architectures

Just need to set model_local to be the desired architecture. 

In the example here, the architecture is imported from an external file in the line:
from resnet18flex import ResNet18Gray
and then instantiated with the line
model_local = ResNet18Gray(num_classes=num_classes, input_height=128)


* Features: 15-second mel spectrograms, augmented with pitch shift and pink noise
* Architecture: ResNet18 (modified for our inputs)
* Inputs: A variety of spectrograms of dimension 12x256, 32x256, and 128x256 (also graphs of dimension 300x300)
    * We'll adjust the ResNet18 architecture use stride=1 in conv1 and skip maxpool when the height is 12 or 32
* Model: Tuning from scratch
* Loss function: Soft Labeling Loss
* Target: ~250 musicmap genres

In [1]:
# Run this command if having trouble with import statements
# %pip install scikit-learn seaborn beautifulsoup4 pyvis

In [2]:
import sys
print(sys.executable)

/opt/homebrew/anaconda3/envs/deep-learning/bin/python


<h2>Imports and Parameters</h2>

In [1]:
# Imports
import os

from PIL import Image

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn.functional import one_hot
from torch import nn, optim
from torch.optim.swa_utils import AveragedModel
from torch.cuda.amp import GradScaler
from torch.utils.data import DataLoader

import torchvision
from torchvision import transforms, datasets

import numpy as np

from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix
from sklearn.utils.multiclass import unique_labels

import matplotlib.pyplot as plt
import seaborn as sns

import pandas as pd
import networkx as nx
from bs4 import BeautifulSoup
from collections import defaultdict
from pyvis.network import Network
from tabulate import tabulate

from tqdm import tqdm

from musicmap_graph_creator import create_musicmap_graph, compute_shortest_paths_between_classes_nx
from resnet18flex import ResNet18Gray
from musicmap_helpers import compute_image_dataset_stats, check_set_gpu, batch_soft_labeling_loss, top_k_accuracy, evaluate, plot_metrics, plot_cm_super


In [2]:
train_data_directory = "./15_second_features_augmented/musicmap_processed_output_splits_train/mel_db_gray"
val_data_directory = "./15_second_features_augmented/musicmap_processed_output_splits_val/mel_db_gray"

# Set the device to the local GPU if available
device = check_set_gpu()

num_epochs = 100  # Maximum number of epochs to try
beta = 5        # Hyperparameter for soft loss function
patience = 10   # Number of epochs in early stopping condition
swa_start_epoch = 5  # start averaging after this epoch

# Edge weights for the graph
primary_edge_weight = 1
secondary_edge_weight = 2
backlash_edge_weight = 3
supergenre_edge_weight = 4
world_supergenre_weight = 1
world_cluster_weight = 1
util_genre_weight = 2
cluster_weight = 1

Using MPS: True


<h2>Shortest Path Matrix</h2>

In [3]:
musicmap_graph = create_musicmap_graph(
    primary_edge_weight, 
    secondary_edge_weight,
    backlash_edge_weight,
    supergenre_edge_weight,
    world_supergenre_weight,
    world_cluster_weight,
    util_genre_weight,
    cluster_weight
)

shortest_graph, class_names = compute_shortest_paths_between_classes_nx(
    class_dir=train_data_directory,  # path to folder with class folders
    graph=musicmap_graph,
    return_tensor=True
)

<h2>Transform</h2>

We'll use the means and standard deviations to normalize the images.

Using ImageFolder might default to loading images as RGB, even if they're just greyscale. So we'll force greyscale as part of the transform for greyscale images.

For use with images that require color, there is also a RGB version of the transform that only strips out the alpha channel.

In [4]:
image_dir = train_data_directory
means, sds = compute_image_dataset_stats(image_dir)

In [5]:
transform_greyscale = transforms.Compose([
    transforms.ToTensor(), # Should always be the last step before feeding into a model
    transforms.Grayscale(num_output_channels=1),  # force grayscale
    transforms.Normalize(mean=means, std=sds)    # Normalize to imagenet mean and standard deviation
])

transform_rgb = transforms.Compose([
    transforms.ToTensor(), # Should always be the last step before feeding into a model
    transforms.Lambda(lambda x: x[:3, :, :]),  # Remove the alpha channel and keep only the first 3 channels (RGB)
    transforms.Normalize(mean=means, std=sds)    # Normalize to imagenet mean and standard deviation
])

<h2>Loading the Data</h2>

Define the datasets and create dataloaders.
ImageFolder automatically creates labels from the folder structure.

In [6]:
train_dataset = torchvision.datasets.ImageFolder(
    root=train_data_directory,
    transform=transform_greyscale
)

val_dataset = torchvision.datasets.ImageFolder(
    root=val_data_directory,
    transform=transform_greyscale
)

In [7]:
train_dataloader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=64,  # Adjust batch size as needed
    shuffle=True,
    num_workers=4,  # Adjust this to tweak multiprocessing
    pin_memory=True,
    prefetch_factor=2
)

val_dataloader = torch.utils.data.DataLoader(
    val_dataset,
    batch_size=10,  # Adjust batch size as needed
    shuffle=False, #Setting to false keeps evaluation stable across epochs
    num_workers=4,
    pin_memory=True,
    prefetch_factor=2 
)

<h2>Architecture</h2>

Compute the number of classes and instantiate the model

In [8]:
num_classes = len(train_dataloader.dataset.classes) 

model_local = ResNet18Gray(num_classes=num_classes, input_height=128)
model_local = model_local.to(device)

<h2>Criterion, Optimizer, and Scheduler</h2>

In [9]:
criterion = lambda outputs, labels: batch_soft_labeling_loss(outputs, labels, shortest_graph, beta)

optimizer = optim.Adam(model_local.parameters(), lr=0.001, weight_decay=1e-4)

scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, mode='max', factor=0.5, patience=2)#, verbose=True)

<h2>Training</h2>

This calls an evaluate function, which automatically saves a confusion matrix.

Incorporates auto-stopping

Automatically saves a best model, a SWA model, a plot of evaluation metrics, and a plot of a confusion matrix.

In [ ]:
# Create index-to-supergenre lookup once
idx_to_supergenre = np.array([cls[:3] for cls in train_dataset.classes])  # shape (num_classes,)

# The line below might speed up training on MPS
torch.set_float32_matmul_precision("high")  # or "medium"

# Metric lists
val_accuracy_list, top3_acc_list, top5_acc_list = [], [], []
precision_list, recall_list, f1_list = [], [], []
mean_dist_list, super_acc_list, super_top3_acc_list = [], [], []
train_loss_list, train_accuracy_list, val_loss_list = [], [], []

# Early stopping and SWA setup
# Going to stop based on mean distance
best_mean_distance = float('inf')
best_model_state = None
counter = 0
swa_model = AveragedModel(model_local)  # model to store average weights


# === Training Loop ===
for epoch in range(num_epochs):
    model_local.train()
    train_loss, correct, total = 0.0, 0, 0


    # Train
    for inputs, labels in tqdm(train_dataloader, desc=f"Epoch {epoch+1} - Training", leave=True, ncols=100):
    #for batch_idx, (inputs, labels) in enumerate(train_dataloader):
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()

        # Putting this inside autocast might speed up cuda computations
        if torch.cuda.is_available():
            with torch.amp.autocast('cuda'):
                outputs = model_local(inputs)

                # Take the log softmax of outputs
                outputs = torch.log_softmax(outputs, dim=1)
                loss = criterion(outputs, labels)
        else:
            outputs = model_local(inputs)
            outputs = torch.log_softmax(outputs, dim=1)
            loss = criterion(outputs, labels)

        
        # On Apple Silicon, just call backward and step
        loss.backward()
        optimizer.step()
        # On CUDA, substitute with the scaler
        #scaler.scale(loss).backward()
        #scaler.step(optimizer)
        #scaler.update()

        train_loss += loss.item()
        _, predicted = outputs.max(1)
        correct += predicted.eq(labels).sum().item()
        total += labels.size(0)

    avg_train_loss = train_loss / len(train_dataloader)
    train_accuracy = correct / total

    # Add train loss and accuracy to their lists
    train_loss_list.append(avg_train_loss)
    train_accuracy_list.append(train_accuracy)

    # --- Validation ---
    metrics = evaluate(idx_to_supergenre, model_local, val_dataloader, criterion, device, shortest_graph, train_dataset)#, epoch=epoch)

    # Early stopping based on mean distance
    if metrics['mean_distance'] < best_mean_distance:
        best_mean_distance = metrics['mean_distance']
        best_model_state = model_local.state_dict()
        counter = 0
        print(f"[Epoch {epoch+1}] New best mean_distance: {best_mean_distance:.4f}")
    else:
        counter += 1
        print(f"No improvement in mean_distance for {counter} epoch(s).")


    # SWA Averaging
    if epoch >= swa_start_epoch:
        swa_model.update_parameters(model_local)

    # Append metrics to lists
    for lst, key in zip(
        [val_loss_list, val_accuracy_list, top3_acc_list, top5_acc_list, precision_list, recall_list, f1_list,
        mean_dist_list, super_acc_list, super_top3_acc_list],
        ["val_loss", "val_accuracy", "top3_accuracy", "top5_accuracy", "precision", "recall", "f1",
        "mean_distance", "supergenre_accuracy", "supergenre_top3_accuracy"]
    ):
        lst.append(metrics[key])

    # Print metrics table
    headers = ["Epoch", "Train Loss", "Train Acc", "Val Loss", "Val Acc",
               "Top-3 Acc", "Top-5 Acc", "Precision", "Recall", "F1",
               "Supergenre Acc", "Top-3 Super", "Mean Dist"]

    row = [epoch + 1,
        f"{avg_train_loss:.4f}", f"{train_accuracy:.2%}",
        f"{metrics['val_loss']:.4f}", f"{metrics['val_accuracy']:.2%}",
        f"{metrics['top3_accuracy']:.2%}", f"{metrics['top5_accuracy']:.2%}",
        f"{metrics['precision']:.2%}", f"{metrics['recall']:.2%}", f"{metrics['f1']:.2%}",
        f"{metrics['supergenre_accuracy']:.2%}", f"{metrics['supergenre_top3_accuracy']:.2%}",
        f"{metrics['mean_distance']:.4f}"]

    try:
        print(tabulate([row], headers=headers, tablefmt="grid"))
    except Exception as e:
        print(f"Error printing metrics: {e}", flush=True)
        print(f"metrics dict: {metrics}")
        raise

    if counter >= patience:
        print(f"Early stopping triggered after {patience} epochs of no improvement.")
        break
    
    scheduler.step(metrics['mean_distance'])

# Save the best model
if best_model_state is not None:
    torch.save(best_model_state, 'best_model.pth')
    print(f"Best model saved with mean_distance: {best_mean_distance:.4f}")

# Save the SWA model
torch.save(swa_model.module.state_dict(), 'swa_model.pth')
print("SWA model saved from averaged checkpoints.")

metrics_df = pd.DataFrame({
    "epoch": list(range(1, len(train_loss_list) + 1)),
    "train_loss": train_loss_list,
    "train_accuracy": train_accuracy_list,
    "val_loss": val_loss_list,
    "val_accuracy": val_accuracy_list,
    "top3_accuracy": top3_acc_list,
    "top5_accuracy": top5_acc_list,
    "precision": precision_list,
    "recall": recall_list,
    "f1": f1_list,
    "supergenre_accuracy": super_acc_list,
    "supergenre_top3_accuracy": super_top3_acc_list,
    "mean_distance": mean_dist_list
})
metrics_df.to_csv("training_metrics.csv", index=False)

# Save graphs of the various metrics
plot_metrics(val_accuracy_list, top3_acc_list, top5_acc_list, precision_list, recall_list, f1_list, mean_dist_list, super_acc_list, super_top3_acc_list, "metrics.png")

# Save the final confusion matrices
cm_genre = metrics['genre_confusion_matrix']
cm_genre.to_csv("confusion_matrix_genre.csv")

cm_supergenre = metrics['supergenre_confusion_matrix']
cm_supergenre.to_csv("confusion_matrix_supergenre.csv")

# Load confusion_matrix.csv and make a heatmap of the confusion matrix
plot_cm_super(cm_supergenre, "supergenre_confusion_matrix.png")

Epoch 1 - Training: 100%|█████████████████████████████████████████| 630/630 [15:57<00:00,  1.52s/it]


[Epoch 1] New best mean_distance: 3.3128
+---------+--------------+-------------+------------+-----------+-------------+-------------+-------------+----------+-------+------------------+---------------+-------------+
|   Epoch |   Train Loss | Train Acc   |   Val Loss | Val Acc   | Top-3 Acc   | Top-5 Acc   | Precision   | Recall   | F1    | Supergenre Acc   | Top-3 Super   |   Mean Dist |
+=========+==============+=============+============+===========+=============+=============+=============+==========+=======+==================+===============+=============+
|       1 |       4.2548 | 9.24%       |     5.0472 | 5.68%     | 13.41%      | 19.27%      | 4.29%       | 6.02%    | 3.48% | 18.98%           | 40.60%        |      3.3128 |
+---------+--------------+-------------+------------+-----------+-------------+-------------+-------------+----------+-------+------------------+---------------+-------------+


Epoch 2 - Training: 100%|█████████████████████████████████████████| 630/630 [16:29<00:00,  1.57s/it]


[Epoch 2] New best mean_distance: 3.2298
+---------+--------------+-------------+------------+-----------+-------------+-------------+-------------+----------+-------+------------------+---------------+-------------+
|   Epoch |   Train Loss | Train Acc   |   Val Loss | Val Acc   | Top-3 Acc   | Top-5 Acc   | Precision   | Recall   | F1    | Supergenre Acc   | Top-3 Super   |   Mean Dist |
+=========+==============+=============+============+===========+=============+=============+=============+==========+=======+==================+===============+=============+
|       2 |       2.8437 | 32.33%      |     5.1704 | 8.47%     | 18.83%      | 25.55%      | 8.55%       | 8.80%    | 6.98% | 24.98%           | 45.11%        |      3.2298 |
+---------+--------------+-------------+------------+-----------+-------------+-------------+-------------+----------+-------+------------------+---------------+-------------+


Epoch 3 - Training: 100%|█████████████████████████████████████████| 630/630 [17:17<00:00,  1.65s/it]


...

Epoch 40 - Training: 100%|████████████████████████████████████████| 630/630 [18:03<00:00,  1.72s/it]


No improvement in mean_distance for 9 epoch(s).
+---------+--------------+-------------+------------+-----------+-------------+-------------+-------------+----------+--------+------------------+---------------+-------------+
|   Epoch |   Train Loss | Train Acc   |   Val Loss | Val Acc   | Top-3 Acc   | Top-5 Acc   | Precision   | Recall   | F1     | Supergenre Acc   | Top-3 Super   |   Mean Dist |
+=========+==============+=============+============+===========+=============+=============+=============+==========+========+==================+===============+=============+
|      40 |        0.159 | 99.93%      |     4.6559 | 11.65%    | 22.88%      | 30.23%      | 10.63%      | 11.72%   | 10.51% | 30.55%           | 48.94%        |      2.8872 |
+---------+--------------+-------------+------------+-----------+-------------+-------------+-------------+----------+--------+------------------+---------------+-------------+


Epoch 41 - Training: 100%|████████████████████████████████████████| 630/630 [18:07<00:00,  1.73s/it]


No improvement in mean_distance for 10 epoch(s).
+---------+--------------+-------------+------------+-----------+-------------+-------------+-------------+----------+--------+------------------+---------------+-------------+
|   Epoch |   Train Loss | Train Acc   |   Val Loss | Val Acc   | Top-3 Acc   | Top-5 Acc   | Precision   | Recall   | F1     | Supergenre Acc   | Top-3 Super   |   Mean Dist |
+=========+==============+=============+============+===========+=============+=============+=============+==========+========+==================+===============+=============+
|      41 |        0.159 | 99.91%      |     4.6412 | 11.54%    | 22.53%      | 29.98%      | 10.66%      | 11.53%   | 10.43% | 30.39%           | 48.45%        |      2.9085 |
+---------+--------------+-------------+------------+-----------+-------------+-------------+-------------+----------+--------+------------------+---------------+-------------+
Early stopping triggered after 10 epochs of no improvement.
Best m

NameError: name 'output_dir' is not defined

In [11]:
metrics_df.to_csv("training_metrics.csv", index=False)

# Save graphs of the various metrics
plot_metrics(val_accuracy_list, top3_acc_list, top5_acc_list, precision_list, recall_list, f1_list, mean_dist_list, super_acc_list, super_top3_acc_list, "metrics.png")

# Save the final confusion matrices
cm_genre = metrics['genre_confusion_matrix']
cm_genre.to_csv("confusion_matrix_genre.csv")

cm_supergenre = metrics['supergenre_confusion_matrix']
cm_supergenre.to_csv("confusion_matrix_supergenre.csv")

# Load confusion_matrix.csv and make a heatmap of the confusion matrix
plot_cm_super(cm_supergenre, "supergenre_confusion_matrix.png")